<h1>Driver Drowsiness Detection</h1><hr>

## Main 

In [1]:
import os

In [2]:
import cv2    #version 3.4.1

In [1]:
import time

In [2]:
import numpy as np   #version 1.18.1

In [3]:
from keras.models import load_model  #version 2.3.1 using tensorflow as backend

In [4]:
from pygame import mixer   #version 1.9.6

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
mixer.init()  #initializing mixer  
sound = mixer.Sound('C:\\Users\\20CP010_hitesh\\Downloads\\drowsiness_project_python-master\\drowsiness_project_python-master\\alarm.wav')  #alarm.wav file for ringing alarm

In [6]:
face = cv2.CascadeClassifier('C:\\Users\\20CP010_hitesh\\Downloads\\drowsiness_project_python-master\\drowsiness_project_python-master\\haar cascade files\\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('C:\\Users\\20CP010_hitesh\\Downloads\\drowsiness_project_python-master\\drowsiness_project_python-master\\haar cascade files\\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('C:\\Users\\20CP010_hitesh\\Downloads\\drowsiness_project_python-master\\drowsiness_project_python-master\\haar cascade files\haarcascade_righteye_2splits.xml')
#these xml files are downloaded from google and used for detecting face , left eye and right eye

NameError: name 'cv2' is not defined

In [ ]:
lbl=['Closed','Open'] 
model = load_model('C:\\Users\\20CP010_hitesh\\Downloads\\drowsiness_project_python-master\\drowsiness_project_python-master\\models\\myModel.h5')  #loading our model
path = os.getcwd() #it will give the path of current working directory
font = cv2.FONT_HERSHEY_COMPLEX_SMALL #font style for putting text on frame
count=0
score=0 #if score goes greater than 15 then ring the alarm
thicc=2 #border width
rpred=[99] #for taking class prediction of right eye
lpred=[99] #for taking class prediction of left eye

In [ ]:
cap = cv2.VideoCapture(0) #initializing video object

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while(True):
    ret, frame = cap.read() #capturing frame by frame
    
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    height,width = frame.shape[:2] #height and width of frame

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #converting frame to gray scale
    
    #now we detect face , left_eye and right_eye , see dataset.ipynb for more information
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    #drawing ractangle of frame size
    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces: #drawing ractangle around the face where x and y are coordinates of upper left corner, w=width, h-height
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]  #extracting right eye from frame
        count=count+1 
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY) #converting to gray scale
        r_eye = cv2.resize(r_eye,(24,24)) #resizing image to (24,24) which input size for our model
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = np.argmax(model.predict(r_eye), axis=-1) #it will predict the class of the image means either eye is open or closed
        if(rpred[0]==1): #if it gives 1 then eye is open because 1 is assigned to open (check model.ipynb)
            lbl='Open'  #set label to open
        if(rpred[0]==0): #if it gives 0 then eye is closed bacause 0 is assigned to closed
            lbl='Closed' #set label to closed
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w] #rxtracting left eye from frame
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY) #converting to gray scale
        l_eye = cv2.resize(l_eye,(24,24)) #resizing image
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = np.argmax(model.predict(r_eye), axis=-1)
        if(lpred[0]==1): #if it gives 1 then eye is open because 1 is assigned to open (check model.ipynb)
            lbl='Open'   #set label to open
        if(lpred[0]==0): #if it gives 0 then eye is closed bacause 0 is assigned to closed
            lbl='Closed' #set label to closed
        break

    if(rpred[0]==0 and lpred[0]==0): #if both eyes are closed incerement the score
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        #it will put the text closed in frame, we have given coordinated of upper left corner of block,font type,color,line type
    # if(rpred[0]==1 or lpred[0]==1):
    else: #if eyes are not closed decrement the score
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0): #restricting our score not to go less than zero
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA) #puting score to frame
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame) #saving image to specified path
        try:
            sound.play() #playing alarm file
            
        except:  # isplaying = False
            pass
        if(thicc<16): #this is for displaying motion of borders when alarm beeps
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) #drawing ractangle
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): #if q is pressed break the loop
        break
#release video object and distroy all the windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


In [ ]:
! pip install opencv-python

In [ ]:
! pip install tensorflow 

In [ ]:
! pip install keras

In [ ]:
! pip install pygame